In [1]:
import pandas as pd
import string
import nltk
import numpy as np
import keras
from tqdm._tqdm_notebook import tqdm_notebook
tqdm_notebook.pandas()
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet
from sklearn.feature_extraction.text import TfidfVectorizer
from keras.models import Sequential
import pickle
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential, Model
from keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation, TimeDistributed, Bidirectional
from keras.layers.embeddings import Embedding
from keras.layers.normalization import BatchNormalization
from keras import optimizers
from keras.layers import Input
from keras.callbacks import EarlyStopping

Using TensorFlow backend.


# Start here

In [2]:
#Save checkpoint
# data.to_csv(r"E:\Yelp\Unfiltered Data\YelpZip\Customs\checkpoint", header=True, index=False)

#Use if kernel crashes
data = pd.read_csv(r"E:\Yelp\Unfiltered Data\YelpZip\Customs\checkpoint")
data.head()

,index,user,item,review,rating,real,date
0,0,5044,0,drink be bad the hot chocolate be water down a...,1.0,-1,2014-11-16
1,1,5045,0,this be the bad experience ive ever have a cas...,1.0,-1,2014-09-08
2,2,5046,0,this be locate on the site of the old spruce s...,3.0,-1,2013-10-06
3,3,5047,0,i enjoy coffee and breakfast twice at toast du...,5.0,-1,2014-11-30
4,4,5048,0,i love toast the food choice be fantastic i lo...,5.0,-1,2014-08-28


In [3]:
data['review'] = data['review'].apply(lambda s: str(s))

In [4]:
#1 for good
#0 for bad
def binarize(n):
    if n==-1:
        return 1
    else:
        return 0

data['real'] = data['real'].apply(lambda s: binarize(s))

In [5]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data['review'], data['real'], test_size=0.30, random_state=42)

In [6]:
data.groupby('real').count()

,index,user,item,review,rating,date
real,,,,,,
0,528019,528019,528019,528019,528019,528019
1,80439,80439,80439,80439,80439,80439


In [7]:
tokenizer = Tokenizer(num_words= 2000)
tokenizer.fit_on_texts(data['review'])
sequences = tokenizer.texts_to_sequences(X_train)
data1 = pad_sequences(sequences, maxlen=100)

In [8]:
# inputs = Input(shape=[100])
# layers = Embedding(2000,32,input_length=100)(inputs)
# layers = LSTM(128, recurrent_dropout=0.2)(layers)
# layers = Dense(1)(layers)
# layers = Activation('sigmoid')(layers)
# model = Model(inputs=inputs,outputs=layers)
# model.compile(loss='binary_crossentropy',optimizer='rmsprop',metrics=['accuracy'])

# def create_conv_model():
#     model_conv = Sequential()
#     model_conv.add(Embedding(vocabulary_size, 100, input_length=50))
#     model_conv.add(Dropout(0.2))
#     model_conv.add(Conv1D(64, 5, activation='relu'))
#     model_conv.add(MaxPooling1D(pool_size=4))
#     model_conv.add(LSTM(100))
#     model_conv.add(Dense(1, activation='sigmoid'))
#     model_conv.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
#     return model_conv


model = Sequential()
model.add(Embedding(2000, 32, input_length=100))
model.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(BatchNormalization())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(BatchNormalization())
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 100, 32)           64000     
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 100, 32)           3104      
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 50, 32)            0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 50, 32)            128       
_________________________________________________________________
dropout_1 (Dropout)          (None, 50, 32)            0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 50, 32)            3104      
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 25, 32)            0         
__________

In [9]:
model.fit(data1,y_train,batch_size=1000,epochs=1, verbose = 1)

Epoch 1/1


194000/425920 [============>.................] - ETA: 3:09:59 - loss: 0.9111 - acc: 0.48 - ETA: 2:59:31 - loss: 0.8804 - acc: 0.49 - ETA: 3:01:22 - loss: 0.8737 - acc: 0.50 - ETA: 2:59:15 - loss: 0.8628 - acc: 0.50 - ETA: 2:57:48 - loss: 0.8462 - acc: 0.51 - ETA: 2:56:11 - loss: 0.8370 - acc: 0.51 - ETA: 2:56:01 - loss: 0.8289 - acc: 0.52 - ETA: 2:54:24 - loss: 0.8273 - acc: 0.52 - ETA: 2:54:59 - loss: 0.8194 - acc: 0.53 - ETA: 2:53:28 - loss: 0.8155 - acc: 0.53 - ETA: 2:53:28 - loss: 0.8101 - acc: 0.53 - ETA: 2:53:22 - loss: 0.8042 - acc: 0.53 - ETA: 2:52:26 - loss: 0.7976 - acc: 0.54 - ETA: 2:51:17 - loss: 0.7931 - acc: 0.54 - ETA: 2:51:02 - loss: 0.7898 - acc: 0.54 - ETA: 2:51:15 - loss: 0.7848 - acc: 0.54 - ETA: 2:50:58 - loss: 0.7814 - acc: 0.54 - ETA: 2:50:49 - loss: 0.7781 - acc: 0.54 - ETA: 2:51:57 - loss: 0.7740 - acc: 0.55 - ETA: 2:52:28 - loss: 0.7707 - acc: 0.55 - ETA: 2:51:53 - loss: 0.7671 - acc: 0.55 - ETA: 2:51:33 - loss: 0.7641 - acc: 0.55 - ETA: 2:50:55 - loss: 0.7605

425920/425920 [==============================] - ETA: 13:05 - loss: 0.4479 - acc: 0.81 - ETA: 12:39 - loss: 0.4477 - acc: 0.82 - ETA: 12:14 - loss: 0.4474 - acc: 0.82 - ETA: 11:49 - loss: 0.4473 - acc: 0.82 - ETA: 11:23 - loss: 0.4471 - acc: 0.82 - ETA: 10:58 - loss: 0.4469 - acc: 0.82 - ETA: 10:33 - loss: 0.4468 - acc: 0.82 - ETA: 10:07 - loss: 0.4466 - acc: 0.82 - ETA: 9:42 - loss: 0.4463 - acc: 0.8208 - ETA: 9:16 - loss: 0.4462 - acc: 0.820 - ETA: 8:51 - loss: 0.4460 - acc: 0.821 - ETA: 8:25 - loss: 0.4457 - acc: 0.821 - ETA: 8:00 - loss: 0.4456 - acc: 0.821 - ETA: 7:35 - loss: 0.4455 - acc: 0.821 - ETA: 7:09 - loss: 0.4452 - acc: 0.821 - ETA: 6:44 - loss: 0.4450 - acc: 0.821 - ETA: 6:18 - loss: 0.4448 - acc: 0.821 - ETA: 5:53 - loss: 0.4447 - acc: 0.821 - ETA: 5:28 - loss: 0.4446 - acc: 0.821 - ETA: 5:02 - loss: 0.4444 - acc: 0.821 - ETA: 4:37 - loss: 0.4443 - acc: 0.822 - ETA: 4:11 - loss: 0.4441 - acc: 0.822 - ETA: 3:46 - loss: 0.4439 - acc: 0.822 - ETA: 3:21 - loss: 0.4438 - acc

In [10]:
test_sequences = tokenizer.texts_to_sequences(X_test)
test_sequences_matrix = pad_sequences(test_sequences,maxlen=100)

In [11]:
# accr = model.evaluate(test_sequences_matrix,y_test)
y_pred = model.predict(test_sequences_matrix)

In [17]:
tst = []
for i in y_pred:
    if i >= 0.60:
        tst.append(1)
    else:
        tst.append(0)

from sklearn.metrics import classification_report
print(classification_report(y_test, tst))

D:\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


             precision    recall  f1-score   support

          0       0.87      1.00      0.93    158422
          1       0.00      0.00      0.00     24116

avg / total       0.75      0.87      0.81    182538



In [13]:
results = model.evaluate(test_sequences_matrix, y_test)

182538/182538 [==============================] - ETA: 14:5 - ETA: 1:0 - ETA: 42s - ETA: 34 - ETA: 29 - ETA: 26 - ETA: 24 - ETA: 23 - ETA: 22 - ETA: 21 - ETA: 21 - ETA: 20 - ETA: 20 - ETA: 19 - ETA: 19 - ETA: 19 - ETA: 18 - ETA: 18 - ETA: 18 - ETA: 18 - ETA: 17 - ETA: 17 - ETA: 17 - ETA: 17 - ETA: 17 - ETA: 17 - ETA: 17 - ETA: 17 - ETA: 16 - ETA: 16 - ETA: 16 - ETA: 16 - ETA: 16 - ETA: 16 - ETA: 16 - ETA: 16 - ETA: 16 - ETA: 16 - ETA: 15 - ETA: 15 - ETA: 15 - ETA: 15 - ETA: 15 - ETA: 15 - ETA: 15 - ETA: 15 - ETA: 15 - ETA: 15 - ETA: 15 - ETA: 15 - ETA: 14 - ETA: 14 - ETA: 14 - ETA: 14 - ETA: 14 - ETA: 14 - ETA: 14 - ETA: 14 - ETA: 14 - ETA: 14 - ETA: 14 - ETA: 14 - ETA: 14 - ETA: 14 - ETA: 13 - ETA: 13 - ETA: 13 - ETA: 13 - ETA: 13 - ETA: 13 - ETA: 13 - ETA: 13 - ETA: 13 - ETA: 13 - ETA: 13 - ETA: 13 - ETA: 13 - ETA: 13 - ETA: 13 - ETA: 12 - ETA: 12 - ETA: 12 - ETA: 12 - ETA: 12 - ETA: 12 - ETA: 12 - ETA: 12 - ETA: 12 - ETA: 12 - ETA: 12 - ETA: 12 - ETA: 12 - ETA: 12 - ETA: 12 - ETA: 12

In [14]:
print("Accuracy:", results[1]*100, "%")

Accuracy: 86.78850431149635 %


In [34]:
b/a

0.14800052828693527